In [40]:
import numpy as np
import pandas as pd
import pickle
from pyproj import Proj, transform
from bokeh.plotting import Figure, figure, output_file, show
from bokeh.models import WMTSTileSource, ColumnDataSource

In [36]:
""" Problem 1 """
accidents = pd.read_pickle("/Users/Deehrlich/Desktop/BootCamp2017/Computation/Wk2_VisPandas/fars_data/final_accidents2.pickle")
drivers = pd.read_pickle("/Users/Deehrlich/Desktop/BootCamp2017/Computation/Wk2_VisPandas/fars_data/final_drivers.pickle")

In [37]:
""" Problem 2 """
from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    Inputs:
    longitudes (array-like) : array of longitudes
    latitudes (array-like) : array of latitudes
    Example:
    x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals

accidents["x"], accidents["y"] = convert(accidents.LONGITUD, accidents.LATITUDE)

In [38]:
""" Problem 4 """

fig = Figure(plot_width=1100, plot_height=650,
    x_range=(-13000000, -7000000), y_range=(2750000, 6250000),
    tools=["wheel_zoom", "pan"], active_scroll="wheel_zoom", webgl=True)
fig.axis.visible = False

STAMEN_TONER_BACKGROUND = WMTSTileSource(
    url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
    attribution=(
        'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
        'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
        'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
        'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
    )
)

fig.add_tile(STAMEN_TONER_BACKGROUND)

TileRenderer(id='18df2f37-df57-46e9-815d-dbdeaf6bf4ba', ...)

In [41]:
""" Problem 6 """

#there is an overlap in drunk and speeding
drunk = accidents[(accidents['DRUNK_DR'] == 1)]
speeding = accidents[accidents["SP"]==1]
both = accidents[(accidents['DRUNK_DR'] == 1)&(accidents['SP'] == 1)]
other = accidents[(accidents['DRUNK_DR'] == 0) & (accidents['SP'] == 0)]

#ColumnDataSource
circ_source = ColumnDataSource(drunk)
circ_drunk = fig.circle(x='x', y='y', source = circ_source, fill_color = 'b')

circ_source = ColumnDataSource(speeding)
circ_speed = fig.circle(x='x', y='y', source = circ_source, fill_color = 'g')

circ_source = ColumnDataSource(both)
circ_both = fig.circle(x='x', y='y', source = circ_source, fill_color = 'r')

circ_source = ColumnDataSource(other)
circ_other = fig.circle(x='x', y='y', source = circ_source, fill_color = 'k')

show(fig)